In [44]:
# Initial Imports

import pandas as pd
import numpy as np
import requests
import json
import datetime as dt
from textblob import TextBlob
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
# Import flair pre-trained sentiment model
from flair.models import TextClassifier
classifier = TextClassifier.load('en-sentiment')
# Import flair Sentence to process input text
from flair.data import Sentence
import regex as re


pd.set_option('display.max_colwidth', 1000)

2022-01-12 19:30:51,067 loading file C:\Users\sjufa\.flair\models\sentiment-en-mix-distillbert_4.pt


In [53]:


# Making the API call to retrieve tweets 

url = "https://twitter32.p.rapidapi.com/getSearch"

# These parameters need to be set before running API call... could in the future receive inputs from user
## TODO -- if input is received, datetime will need to be formatted correctly for API call

hashtag = '$TSLA'
start_date = '2021-11-05'
end_date = '2022-01-07'

querystring = {"hashtag": hashtag, "start_date": start_date,"end_date": end_date,"lang":"en"}

headers = {
    'x-rapidapi-host': "twitter32.p.rapidapi.com",
    'x-rapidapi-key': "49e5cedc9fmsh6a2df83dacfc4c1p1c3469jsn3edb2e8cb9df"
    }

response = requests.get(url, headers=headers, params=querystring).json()
df = pd.DataFrame(response['data']['tweets']).T
df

,created_at,id,id_str,full_text,truncated,display_text_range,entities,extended_entities,source,in_reply_to_status_id,...,possibly_sensitive,possibly_sensitive_editable,lang,supplemental_language,self_thread,conversation_control,limited_actions,quoted_status_id,quoted_status_id_str,quoted_status_permalink
1478780165127909381,Wed Jan 05 17:27:20 +0000 2022,1478780165127909381,1478780165127909381,GM is so ahead of their schedule. \n\nThey delivered 26 of those 30 EV’s in Q4 ‘21 alone!!! 😳🤯\n\n$TSLA https://t.co/C8YwOTdpO8,False,"[0, 99]","{'hashtags': [], 'symbols': [{'text': 'TSLA', 'indices': [94, 99]}], 'user_mentions': [], 'urls': [], 'media': [{'id': 1478780159465566211, 'id_str': '1478780159465566211', 'indices': [100, 123], 'media_url': 'http://pbs.twimg.com/media/FIWuuFnXMAMWzgC.jpg', 'media_url_https': 'https://pbs.twimg.com/media/FIWuuFnXMAMWzgC.jpg', 'url': 'https://t.co/C8YwOTdpO8', 'display_url': 'pic.twitter.com/C8YwOTdpO8', 'expanded_url': 'https://twitter.com/JorritvdB/status/1478780165127909381/photo/1', 'type': 'photo', 'original_info': {'width': 2048, 'height': 946, 'focus_rects': [{'x': 25, 'y': 0, 'h': 946, 'w': 1689}, {'x': 396, 'y': 0, 'h': 946, 'w': 946}, {'x': 454, 'y': 0, 'h': 946, 'w': 830}, {'x': 633, 'y': 0, 'h': 946, 'w': 473}, {'x': 0, 'y': 0, 'h': 946, 'w': 2048}]}, 'sizes': {'thumb': {'w': 150, 'h': 150, 'resize': 'crop'}, 'small': {'w': 680, 'h': 314, 'resize': 'fit'}, 'medium': {'w': 1200, 'h': 554, 'resize': 'fit'}, 'large': {'w': 2048, 'h': 946, 'resize': 'fit'}}, 'features': {'s...","{'media': [{'id': 1478780159465566211, 'id_str': '1478780159465566211', 'indices': [100, 123], 'media_url': 'http://pbs.twimg.com/media/FIWuuFnXMAMWzgC.jpg', 'media_url_https': 'https://pbs.twimg.com/media/FIWuuFnXMAMWzgC.jpg', 'url': 'https://t.co/C8YwOTdpO8', 'display_url': 'pic.twitter.com/C8YwOTdpO8', 'expanded_url': 'https://twitter.com/JorritvdB/status/1478780165127909381/photo/1', 'type': 'photo', 'original_info': {'width': 2048, 'height': 946, 'focus_rects': [{'x': 25, 'y': 0, 'h': 946, 'w': 1689}, {'x': 396, 'y': 0, 'h': 946, 'w': 946}, {'x': 454, 'y': 0, 'h': 946, 'w': 830}, {'x': 633, 'y': 0, 'h': 946, 'w': 473}, {'x': 0, 'y': 0, 'h': 946, 'w': 2048}]}, 'sizes': {'thumb': {'w': 150, 'h': 150, 'resize': 'crop'}, 'small': {'w': 680, 'h': 314, 'resize': 'fit'}, 'medium': {'w': 1200, 'h': 554, 'resize': 'fit'}, 'large': {'w': 2048, 'h': 946, 'resize': 'fit'}}, 'features': {'small': {'faces': [{'x': 338, 'y': 122, 'h': 29, 'w': 29}]}, 'medium': {'faces': [{'x': 596, 'y': 215,...","<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",None,...,False,True,en,None,NaN,NaN,NaN,NaN,NaN,NaN
1478415419480457218,Tue Jan 04 17:17:58 +0000 2022,1478415419480457218,1478415419480457218,"In 2023, $TSLA will sell 10% of all 🇺🇸 cars",False,"[0, 43]","{'hashtags': [], 'symbols': [{'text': 'TSLA', 'indices': [9, 14]}], 'user_mentions': [], 'urls': []}",NaN,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",None,...,NaN,NaN,en,None,"{'id': 1478415419480457218, 'id_str': '1478415419480457218'}",NaN,NaN,NaN,NaN,NaN
1478591107470614531,Wed Jan 05 04:56:05 +0000 2022,1478591107470614531,1478591107470614531,"In the not too distant future, Elon Musk will be one hated man. \nRight now everyone owns Tesla shares and ignore all his broken promises and fraudulent behavior because ""hey, I'm getting rich"" but when the stock collapses and SpaceX is bankrupt, people will sober up.\n$tsla $tslaq",False,"[0, 280]","{'hashtags': [], 'symbols': [{'text': 'tsla', 'indices': [268, 273]}, {'text': 'tslaq', 'indices': [274, 280]}], 'user_mentions': [], 'urls': []}",NaN,"<a href=""http://twitter.com/download/android"" rel=""nofollow"">Twitter for Android</a>",None,...,NaN,NaN,en,None,NaN,NaN,NaN,NaN,NaN,NaN
1478480813075357700,Tue Jan 04 21:37:49 +0000 2022,1478480813075357700,1478480813075357700,"BREAKING: Mizuho Securities analyst Vijay Rakesh raised their price target on 

In [54]:
# Parsing and formatting the dataframe to index by date created and isolating the 'full_text' and 'retweet_count' columns for sentiment analysis
df_filtered = df.loc[:,["created_at","full_text", 'retweet_count']]
df_filtered.index = pd.to_datetime(df_filtered['created_at'], utc=True)
df_filtered.index.name = 'Date'
df_filtered.drop(columns='created_at', inplace=True)
df_filtered

,full_text,retweet_count
Date,,
2022-01-05 17:27:20+00:00,GM is so ahead of their schedule. \n\nThey delivered 26 of those 30 EV’s in Q4 ‘21 alone!!! 😳🤯\n\n$TSLA https://t.co/C8YwOTdpO8,218
2022-01-04 17:17:58+00:00,"In 2023, $TSLA will sell 10% of all 🇺🇸 cars",42
2022-01-05 04:56:05+00:00,"In the not too distant future, Elon Musk will be one hated man. \nRight now everyone owns Tesla shares and ignore all his broken promises and fraudulent behavior because ""hey, I'm getting rich"" but when the stock collapses and SpaceX is bankrupt, people will sober up.\n$tsla $tslaq",6
2022-01-04 21:37:49+00:00,"BREAKING: Mizuho Securities analyst Vijay Rakesh raised their price target on $TSLA to $1,300 (from $950) while maintaining a Buy rating.",144
2022-01-05 16:46:06+00:00,What happened to the argument that $TSLA loses money on every car it sells? 🤣,107
2022-01-06 18:05:16+00:00,"By the end of this year, we will look back with disbelief at how $TSLA was in the $1050s in Jan.",53
2022-01-04 23:29:26+00:00,$TSLA Raised to $1300 from $950 at Mizuho Securities,61
2022-01-06 15:03:22+00:00,"$TSLA catalysts - 1/6\n1/ FY’22 Vol/EPS increases 1/26\n2/ 4Q EPS/product roadmap 1/26\n3/ Berlin/Austin open Jan\n4/ Inv Grade Upgrade Feb\n5/ Biden $8K EV credit Mar\n6/ FSD beta launch F’22 2Q\n7/ 4680 scale product’n F’22 2H\n8/ CyTruck launch F’22 2H\n9/ Compact launch F’23\n$1,600 PT",218
2022-01-04 20:36:12+00:00,"I don’t get why institutions buy $F over $TSLA. It would be like buying BlackBerry or Nokia instead of $AAPL after iPhone blew away competitors a decade ago. Ford doesn’t hire great engineers, it’s tech platform is woeful, it’s brand is tainted, its mgmt/culture don’t get EVs. https://t.co/PfjiXyH0Oo",219


In [55]:
# We need to clean up our tweets so they can be analyzed in string format

# Create a function to clean the tweets
def clean_tweet(tweet_text): 
    return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", tweet_text).split()) 


#applying this function to Text column of our dataframe
df_filtered["full_text"] = df_filtered["full_text"].apply(clean_tweet)
df_filtered

,full_text,retweet_count
Date,,
2022-01-05 17:27:20+00:00,GM is so ahead of their schedule They delivered 26 of those 30 EV s in Q4 21 alone TSLA,218
2022-01-04 17:17:58+00:00,In 2023 TSLA will sell 10 of all cars,42
2022-01-05 04:56:05+00:00,In the not too distant future Elon Musk will be one hated man Right now everyone owns Tesla shares and ignore all his broken promises and fraudulent behavior because hey I m getting rich but when the stock collapses and SpaceX is bankrupt people will sober up tsla tslaq,6
2022-01-04 21:37:49+00:00,BREAKING Mizuho Securities analyst Vijay Rakesh raised their price target on TSLA to 1 300 from 950 while maintaining a Buy rating,144
2022-01-05 16:46:06+00:00,What happened to the argument that TSLA loses money on every car it sells,107
2022-01-06 18:05:16+00:00,By the end of this year we will look back with disbelief at how TSLA was in the 1050s in Jan,53
2022-01-04 23:29:26+00:00,TSLA Raised to 1300 from 950 at Mizuho Securities,61
2022-01-06 15:03:22+00:00,TSLA catalysts 1 6 1 FY 22 Vol EPS increases 1 26 2 4Q EPS product roadmap 1 26 3 Berlin Austin open Jan 4 Inv Grade Upgrade Feb 5 Biden 8K EV credit Mar 6 FSD beta launch F 22 2Q 7 4680 scale product n F 22 2H 8 CyTruck launch F 22 2H 9 Compact launch F 23 1 600 PT,218
2022-01-04 20:36:12+00:00,I don t get why institutions buy F over TSLA It would be like buying BlackBerry or Nokia instead of AAPL after iPhone blew away competitors a decade ago Ford doesn t hire great engineers it s tech platform is woeful it s brand is tainted its mgmt culture don t get EVs,219


In [56]:
# Define a function to get Flair sentiment prediction score
def score_flair(text):
  sentence = Sentence(text)
  classifier.predict(sentence)
  score = sentence.labels[0].score
  value = sentence.labels[0].value
  return score, value

# Get sentiment score for each review
df_filtered['scores_flair'] = df_filtered['full_text'].apply(lambda s: score_flair(s)[0])
# Predict sentiment label for each review
df_filtered['pred_flair'] = df_filtered['full_text'].apply(lambda s: score_flair(s)[1])
# Check the distribution of the score
df_filtered['scores_flair'].describe()
# Change the label of flair prediction to 0 if negative and 1 if positive
mapping = {'NEGATIVE': 0, 'POSITIVE': 1}
df_filtered['pred_flair_numerical'] = df_filtered['pred_flair'].map(mapping)
df_filtered

,full_text,retweet_count,scores_flair,pred_flair,pred_flair_numerical
Date,,,,,
2022-01-05 17:27:20+00:00,GM is so ahead of their schedule They delivered 26 of those 30 EV s in Q4 21 alone TSLA,218,0.991020,NEGATIVE,0
2022-01-04 17:17:58+00:00,In 2023 TSLA will sell 10 of all cars,42,0.800326,POSITIVE,1
2022-01-05 04:56:05+00:00,In the not too distant future Elon Musk will be one hated man Right now everyone owns Tesla shares and ignore all his broken promises and fraudulent behavior because hey I m getting rich but when the stock collapses and SpaceX is bankrupt people will sober up tsla tslaq,6,0.984531,NEGATIVE,0
2022-01-04 21:37:49+00:00,BREAKING Mizuho Securities analyst Vijay Rakesh raised their price target on TSLA to 1 300 from 950 while maintaining a Buy rating,144,0.527514,POSITIVE,1
2022-01-05 16:46:06+00:00,What happened to the argument that TSLA loses money on every car it sells,107,0.999916,NEGATIVE,0
2022-01-06 18:05:16+00:00,By the end of this year we will look back with disbelief at how TSLA was in the 1050s in Jan,53,0.831751,NEGATIVE,0
2022-01-04 23:29:26+00:00,TSLA Raised to 1300 from 950 at Mizuho Securities,61,0.928677,NEGATIVE,0
2022-01-06 15:03:22+00:00,TSLA catalysts 1 6 1 FY 22 Vol EPS increases 1 26 2 4Q EPS product roadmap 1 26 3 Berlin Austin open Jan 4 Inv Grade Upgrade Feb 5 Biden 8K EV credit Mar 6 FSD beta launch F 22 2Q 7 4680 scale product n F 22 2H 8 CyTruck launch F 22 2H 9 Compact launch F 23 1 600 PT,218,0.999550,POSITIVE,1
2022-01-04 20:36:12+00:00,I don t get why institutions buy F over TSLA It would be like buying BlackBerry or Nokia instead of AAPL after iPhone blew away competitors a decade ago Ford doesn t hire great engineers it s tech platform is woeful it s brand is tainted its mgmt culture don t get EVs,219,0.999324,NEGATIVE,0


In [58]:
#Converting the categorical variable columns into numerical with get_dummies
df_dummies = pd.get_dummies(df_filtered['pred_flair'])
# Dropping the row if the scores_flair confidence level is below 0.75
df_filtered.drop(df_filtered[df_filtered.scores_flair < 0.75].index, inplace=True)
# Creating the 'df_signals' dataframe that we will use to append to our primary dataset, using only the columns with relevant numerical values, resampled by day
df_signals = df_filtered.drop(columns=['scores_flair', 'pred_flair_numerical', 'pred_flair'])
final_df = pd.concat([df_signals, df_dummies]).resample('D').sum()
final_df

,NEGATIVE,POSITIVE
Date,,
2022-01-04 00:00:00+00:00,3.0,3.0
2022-01-05 00:00:00+00:00,3.0,1.0
2022-01-06 00:00:00+00:00,4.0,4.0


### After pulling tweet data, performing sentiment analysis using flair and then filtered and resampling the dataframe, we return a dataframe with datetime indices with two columns -- flair_negative, which is the sum of all negative tweets for the day, and flair_positive, which is the sum of all positive tweets for the day -- at some point we could think about factoring in number of retweets or something along those lines to weigh the tweets by their impact, but the results from the API call already filter by this metric, so could be superfluous.

In [7]:
df_signals

,full_text
Date,
2022-01-06 00:00:00+00:00,Following were the Top 5 tickers mentioned on r wallstreetbets TSLA AMC F SPY DD The data was fetched at 06 01 2022 00 00 02 AM UTC wallstreetbets stocks stonks trading TSLA AMC F SPY DDTSLA 15s delayed Issued Press Release on January 06 08 00 00 Could A Graphite Shortage Derail The 3 Trillion EV Boom
